## Observations and Insights 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import sem
# %matplotlib notebook

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
merge_df = pd.merge(mouse_metadata, study_results, on="Mouse ID")

# Display the data table for preview
merge_df.head()

In [ ]:
# Checking the number of mice.
total_mice = len(merge_df["Mouse ID"].unique())
total_mice_df = pd.DataFrame({"Total Mice": [total_mice]})
total_mice_df

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
dup_mouse = merge_df.loc[merge_df.duplicated(subset = ["Mouse ID", "Timepoint"]),"Mouse ID"].unique()
dup_mouse

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 



In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.

# I need to figure out how to create a clean data frame with all the other columns without dropping necessary data. 
final_df = merge_df[merge_df["Mouse ID"].isin(dup_mouse)==False]
final_df.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
total_mice = len(final_df["Mouse ID"].unique())
total_mice

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straighforward, creating multiple series and putting them all together at the end.

vol_mean = final_df.groupby("Drug Regimen").mean()["Tumor Volume (mm3)"]
vol_median = final_df.groupby("Drug Regimen").median()["Tumor Volume (mm3)"]
vol_std = final_df.groupby("Drug Regimen").std()["Tumor Volume (mm3)"]
vol_var = final_df.groupby("Drug Regimen").var()["Tumor Volume (mm3)"]
vol_sem = final_df.groupby("Drug Regimen").sem()["Tumor Volume (mm3)"]

summary_df = pd.DataFrame({"Mean": vol_mean,
                           "Median": vol_median, 
                           "Variance": vol_var,
                           "Standard Dev.": vol_std,
                           "SEM": vol_sem
                          
                          })
summary_df

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
summary_new = final_df.groupby("Drug Regimen").agg({"Tumor Volume (mm3)":["mean","median","var","std","sem"]})
summary_new
# This method produces everything in a single groupby function


## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pandas. 
# i have to find a way to add the number of mice to a dataframe in a way that will allow me to plot them. 
# Filter the DataFrame down only to those columns to chart
mice_and_drugs = final_df.groupby("Drug Regimen").count()["Mouse ID"]


# Set the index to be "Drug Regimen" so they will be used as labels
# mice_and_drugs = mice_and_drugs.set_index("Drug Regimen")

mice_and_drugs.plot(kind="bar") # figsize=(10,3)

# Set a title for the chart
plt.title("Treatments")

plt.show()
plt.tight_layout()

In [ ]:
# Generate a bar plot showing the total number of mice for each treatment throughout the course of the study using pyplot.
mice_and_drugs = final_df.groupby("Drug Regimen").count()["Mouse ID"]
mice_and_drugs
plt.bar(mice_and_drugs.index, mice_and_drugs.values, color='r', alpha=0.5, align="center")
plt.xticks(rotation=90)

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
count = final_df["Sex"].value_counts()
label_y = ['Male', 'Female']
count.plot(kind="pie", autopct='%1.1f%%')

color1 = ['violet', 'green']
color2 = ['red', 'blue']
explode = [0.1, 0]

plt.show()
plt.tight_layout()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
plt.pie(count, explode=explode, labels=label_y, colors=color2, autopct='%1.1f%%', shadow=True,
       startangle=140)

#add title
plt.title('Male v. Female Population')
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
max_tumor = final_df.groupby("Mouse ID").max()["Timepoint"]

max_tumor = max_tumor.reset_index()
max_tumor

# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
merge_df = pd.merge(max_tumor[["Mouse ID", "Timepoint"]], final_df, how="left", on=["Mouse ID", "Timepoint"])
merge_df

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
drug_list = ["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]
tumor_list = []

for x in drug_list: 
    tumor_vol = merge_df.loc[merge_df["Drug Regimen"]==x]["Tumor Volume (mm3)"]
    tumor_list.append(tumor_vol)
    quartiles = tumor_vol.quantile([.25,.5,.75])
    lowerq = quartiles[.25]
    upperq = quartiles[.5]
    iqr = upperq - lowerq

print(f"The lower quartile of tumor volume is: {lowerq}")
print(f"The upper quartile of tumor volume is: {upperq}")
print(f"The interquartile range of tumor volume is: {iqr}")
print(f"The median of tumor volume is: {quartiles[.5]}")

# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
fig1, ax1 = plt.subplots()
plt.boxplot(tumor_list)
plt.show

## Line and Scatter Plots

In [ ]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin
cap_df = final_df.loc[final_df["Drug Regimen"] == "Capomulin"]
y_axis = cap_df.groupby("Timepoint").mean()["Tumor Volume (mm3)"]
x_axis = cap_df.groupby("Timepoint").count().reset_index()["Timepoint"]
plt.xlabel("Timepoint")
plt.ylabel("Avg. Tumor Volume for Capomulin Treatments")
plt.plot(x_axis, y_axis)
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen
mouse_weight = final_df.groupby("Mouse ID").mean()["Weight (g)"]
tumor_volume = final_df.groupby("Mouse ID").mean()["Tumor Volume (mm3)"]
tumor_volume
plt.scatter(mouse_weight, tumor_volume, marker="o", facecolors="blue", edgecolors="black",
           s=x_axis, alpha=.75)
plt.xlabel("Timepoint")
plt.ylabel("Average Tumor Volume Across Capomulin Treatment")

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
cap_df = final_df[final_df["Drug Regimen"]=="Capomulin"]
cap_df["Weight (g)"].corr(cap_df["Tumor Volume (mm3)"])


In [ ]:
st.linregress(cap_df["Weight (g)"], cap_df["Tumor Volume (mm3)"])
